In [96]:
using DelimitedFiles, Distributions, Plots
pgfplotsx()

Plots.PGFPlotsXBackend()

In [97]:
sunspot = readdlm("../../../data/SN_y_tot_V2.0.csv", ';')
select_1945_2020 = filter((s) -> s[1] > 1945 && s[1] < 2021, collect(eachrow(sunspot)))
observations = map((s) -> round(s[2]), select_1945_2020);


In [98]:
# to check that matched with Semih
cvi = readdlm("../../../estimations/cvi_sunspot_results.csv")
cvmp = readdlm("../../../estimations/cvmp_sunspot_results.csv")
aismp = readdlm("../../../estimations/ais_mp_sunspot_results.csv")
nuts = readdlm("../../../estimations/nuts_sunspot_results.csv");

In [99]:
custom_colors = [:green, :blue, :red]
algorithms = ["AISMP", "CVMP", "SVMP"]
data_list = [aismp, cvmp, cvi]
linestyles = [:dash, :solid, :dash]

estimation_plot = plot(title="Sunspot Recovery Comparison",
                       guidefontsize=14,
                       tickfontsize=12,
                       legendfontsize=12,
                       legend=:topright)

scatter!(estimation_plot, observations, color=:black,
         xtick=([1, 12, 23, 34, 45, 56, 67, 76], [1945, 1956, 1967, 1978, 1989, 2000, 2011, 2020]),
         xlabel="Years", ylabel="Sunspots", label="Observations", left_margin=10Plots.mm, bottom_margin=5Plots.mm)

for (i, (algorithm, data, color, linestyle)) in enumerate(zip(algorithms, data_list, custom_colors, linestyles))
    plot!(estimation_plot, data[:, 1], ribbon=data[:, 2], label=algorithm, lw=2, size=(1000, 400), color=color, linestyle = linestyle, fillalpha=0.15)
end

for (i, (algorithm, data, color, linestyle)) in enumerate(zip(algorithms, data_list, custom_colors, linestyles))
    plot!(data[:, 1], label=false, color=color, linestyle = linestyle)
end

estimation_plot

In [100]:
# to check that matched with Semih
cvi_fe = readdlm("../../../estimations/cvi_fe_sunspot_results.csv")
cvmp_fe = readdlm("../../../estimations/cvmp_fe_sunspot_results.csv")
aismp_fe = readdlm("../../../estimations/ais_mp_fe_results.csv")

1×10 Matrix{Float64}:
 17031.2  9035.33  7171.81  6807.99  …  6699.37  6699.12  6697.37  6696.55

In [101]:
fe_list = [aismp_fe, cvmp_fe, cvi_fe]


fe_plot = plot(xlabel="VMP iterations", ylabel="Free Energy (nats)", guidefontsize=14, tickfontsize=12, legendfontsize=12, legend=:topright)

for (i, (algorithm, data, color, linestyle)) in enumerate(zip(algorithms, fe_list, custom_colors, linestyles))
    plot!(data[1, :], label=algorithm, color=color, linestyle = linestyle)
end

In [102]:
plot(estimation_plot, fe_plot, layout=grid(1, 2, widths=(2/3,1/3)))
savefig("estimation_fe_plot.tex");